In [ ]:
%cd ../..
import time
from os import environ
# ego = Car on drivableRoad,
#         facing Range(-15, 15) deg relative to roadDirection,
#         with visibleDistance 50,
#         with viewAngle 135 deg

# other1 = Car on intersection,
#             facing Range(50, 135) deg relative to ego.heading

# other2 = Car on intersection,
#             facing -1 * Range(50, 135) deg relative to ego.heading

# require abs(relative heading of other1 from other2) > 100 deg
# require (distance from ego to intersectionRegion) < 10

In [ ]:
if environ["AP_PORT"] is None:
    environ["AP_PORT"] = str(input('port'))

In [ ]:
"""
with (
  select *
  from item_traj
  where item_traj.object_type = 'car'
) as cars

select *
from cars as t1
join cars as t2 on t1.cameraId = t2.cameraId
join Cameras as cam on t1.cameraId = Cameras.id
where
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) >= -15 AND
  facingRelative(cam.egoHeading, road_direction(cam.ego_translation), cam.timestamp) <= 15 AND
  DISTANCE(cam.egoTranslation, t1.centroid, cam.timestamp) < 50 AND
  viewAngle(t1.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  DISTANCE(cam.egoTranslation, t2.centroid, cam.timestamp) < 50 AND
  viewAngle(t2.traj, cam.egoHeading, cam.ego_Translation, cam.timestamp) < 135 AND
  contained(t1.traj, road_segment_type("intersection"), cam.timestamp) and
  contained(t2.traj, road_segment_type("intersection"), cam.timestamp) and
  (facingRelative(t1.heading, cam.egoHeading, cam.timestamp) < -50 ANd
  facingRelative(t1.heading, cam.egoHeading, cam.timestamp) > -135) AND
  (facingRelative(t2.heading, cam.egoHeading, cam.timestamp) < -50 AND
  facingRelative(t2.heading, cam.egoHeading, cam.timestamp) > -135) AND
  DISTANCE(cam.egoTranslation, roadSection("intersection"), cam.timestamp) < 10 AND
  (facingRelative(t1.heading, t2.heading, cam.timestamp) > 100 OR
  facingRelative(t1.heading, t2.heading, cam.timestmap) < -100)

"""

In [ ]:
### Prepare the world for queries ###
from apperception.world import empty_world
from apperception.database import database
# directly ingest the mini dataset and boston roadnetwork if needed
# from apperception.new_db import database
# database.reset()
# from benchmarks.ingest_scenic_data import ingest_data
# ingest_data()

name = 'ScenicWorld' # world name
world = empty_world(name=name)

### Query ###
from apperception.utils import F


def pred(obj1, obj2, cam):
    return (
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) >= -15 and
        F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.timestamp), cam.timestamp) <= 15 and
        F.distance(cam.ego, obj1, cam.timestamp) < 50 and
        F.viewAngle(obj1, cam.ego, cam.timestamp) < 135 and
        F.distance(cam.ego, obj2, cam.timestamp) < 50 and
        F.viewAngle(obj2, cam.ego, cam.timestamp) < 135 and
        F.contained(obj1.traj, F.road_segment("intersection"), cam.timestamp) and
        F.contained(obj2.traj, F.road_segment("intersection"), cam.timestamp) and
        (F.facing_relative(obj1, cam.ego, cam.timestamp) < -50 or
            F.facing_relative(obj1, cam.ego, cam.timestamp) > -135) and
        (F.facing_relative(obj2, cam.ego, cam.timestamp) < -50 or
            F.facing_relative(obj2, cam.ego, cam.timestamp) > -135) and
        F.distance(cam.ego, roadSection(obj1.traj, cam.timestamp)) < 10 and
        (F.facing_relative(obj1, obj2, cam.timestamp) > 100 or
            F.facing_relative(obj1, obj2, cam.timestamp) < -100)
    )
world = world.filter("lambda obj1, obj2, cam: " +
    "obj1.object_id != obj2.object_id and " +
    "F.like(obj1.object_type, 'vehicle%') and " +
    "F.like(obj2.object_type, 'vehicle%') and " +
    "F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego, cam.ego)), -15, 15) and " +
    "F.distance(cam.ego, obj1, cam.timestamp) < 50 and " +
    "F.view_angle(obj1, cam.ego, cam.timestamp) < 135 / 2.0 and " +
    "F.distance(cam.ego, obj2, cam.timestamp) < 50 and " +
    "F.view_angle(obj2, cam.ego, cam.timestamp) < 135 / 2.0 and " +
    "F.contained(obj1.traj, F.road_segment('intersection'), cam.timestamp) and " +
    "F.contained(obj2.traj, F.road_segment('intersection'), cam.timestamp) and " +
    "F.angle_between(F.facing_relative(obj1, cam.ego, cam.timestamp), 50, 135) and " +
    "F.angle_between(F.facing_relative(obj2, cam.ego, cam.timestamp), -135, -50) and " +
                     
    # TODO: distance for intersecting region
    # "F.distance(cam.ego, obj1, cam.timestamp) < 10 and " +
    # "F.distance(cam.ego, obj2, cam.timestamp) < 10 and " +
    "F.minDistance(cam.egoTranslation, F.road_segment('intersection')) < 10 and " +
    "F.angle_between(F.facing_relative(obj1, obj2, cam.timestamp), 100, -100) and " +
    # "F.like(cam.filename, '%n008-2018-09-18-12-07-26-0400__CAM_FRONT__1537287358412404.jpg') and " +
    "1 == 1;"
)

start = time.time()

id_time_camId_filename = world.get_id_time_camId_filename(2)

end = time.time()
print(format(end-start))

In [ ]:
database.sql("""
SELECT 
  table_0.itemId, 
  table_1.itemId, 
  cameras.timestamp, 
  cameras.cameraId, 
  cameras.filename,
  minDistance(
        Cameras.egoTranslation, 
        roadSegment('intersection')
      )
FROM 
  (
    SELECT 
      * 
    FROM 
      item_general_trajectory
  ) as table_0 
  JOIN (
    SELECT 
      * 
    FROM 
      item_general_trajectory
  ) as table_1 USING (cameraId) 
  JOIN Cameras USING (cameraId) 
WHERE 
  (
    (table_0.itemId <> table_1.itemId) 
    AND (
      table_0.objectType LIKE 'vehicle%'
    ) 
    AND (
      table_1.objectType LIKE 'vehicle%'
    ) 
    AND angleBetween(
      facingRelative(
        Cameras.egoHeading, 
        roadDirection(
          Cameras.egoTranslation, Cameras.egoHeading
        )
      ), 
      (-15), 
      15
    ) 
    AND (
      distance(
        Cameras.egoTranslation, table_0.trajCentroids, 
        Cameras.timestamp
      )< 50
    ) 
    AND (
      viewAngle(
        table_0.trajCentroids, Cameras.egoHeading, 
        Cameras.egoTranslation, Cameras.timestamp
      )< 135
    ) 
    AND (
      distance(
        Cameras.egoTranslation, table_1.trajCentroids, 
        Cameras.timestamp
      )< 50
    ) 
    AND (
      viewAngle(
        table_1.trajCentroids, Cameras.egoHeading, 
        Cameras.egoTranslation, Cameras.timestamp
      )< 135
    ) 
    AND contained(
      table_0.trajCentroids, 
      roadSegment('intersection'), 
      Cameras.timestamp
    ) 
    AND contained(
      table_1.trajCentroids, 
      roadSegment('intersection'), 
      Cameras.timestamp
    ) 
    AND angleBetween(
      facingRelative(
        table_0.itemHeadings, Cameras.egoHeading, 
        Cameras.timestamp
      ), 
      50, 
      135
    ) 
    AND angleBetween(
      facingRelative(
        table_1.itemHeadings, Cameras.egoHeading, 
        Cameras.timestamp
      ), 
      (-135), 
      (-50)
    ) 
    AND (
      minDistance(
        Cameras.egoTranslation, 
        roadSegment('intersection')
      )< 10
    ) 
    AND angleBetween(
      facingRelative(
        table_0.itemHeadings, table_1.itemHeadings, 
        Cameras.timestamp
      ), 
      100, 
      (-100)
    ) 
    AND (1 = 1)
  )

""")

In [ ]:
# for k in id_time_camId_filename:
#     print(k[-2], str(k[-3]), k[-1])
#     print("                                           ", *k[:-3])

In [ ]:
resultImages = dict() # maping from image -> (itemId, timestamp, camId) that it was found at
for result in id_time_camId_filename:
    itemId1, itemId2, timestamp, camId, filename = result
    filename = filename.split("/")[-1] # use split so that prefix path is not included in filename
    if filename not in resultImages:
        resultImages[filename] = []
    resultImages[filename].append((itemId1, itemId2, timestamp, camId))

In [ ]:
from show_images import show_images
%matplotlib inline

# data_dir =  "data/nuscenes/experiment_data"
data_dir = '/work/apperception/data/nuScenes/full-dataset-v1.0/Trainval/experiment_data'

In [ ]:
show_images(data_dir, list(resultImages.keys()), sample=10)

In [ ]:
expected_imgs = [
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657119112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127612404.jpg',
    'n008-2018-09-18-12-07-26-0400__CAM_FRONT__1537287358412404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657118612404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657127112404.jpg',
    'n008-2018-08-30-15-16-55-0400__CAM_FRONT__1535657128112404.jpg',
]
show_images(data_dir, expected_imgs, sample=10)

In [ ]:
missing = 0
missing_images = []
for expected_img in expected_imgs:
    if expected_img not in resultImages.keys():
        missing += 1
        missing_images.append(expected_img)
print("Percentage of expected results missing from query: ", missing, "/", len(expected_imgs), "=", 100 * missing / len(expected_imgs), "%")
show_images(data_dir, missing_images, sample=10)

In [ ]:
extra = 0
extra_images = {}
for img in resultImages.keys():
    if img not in expected_imgs:
        extra += 1
        extra_images[img] = resultImages[img]
print("Percentage of images in query but not in expected results: ", extra, "/", len(resultImages.keys()), "=", 100 * extra / len(resultImages.keys()), "%")
show_images(data_dir, extra_images.keys(), sample=10)